# Основные импорты

In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from os import listdir
from os.path import isfile, join
import spacy

C:\Users\Lev\PycharmProjects\hack\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Считываем данные

In [2]:
data = pd.read_csv("train_data/dataset.csv")
stds_ls = pd.read_csv("train_data/standarts_list.csv")
stds = pd.read_csv("train_data/standarts.csv")

## Считываем пути к файлам со стандартами

In [3]:
files = [f for f in listdir("files") if isfile(join("files", f))]
files

['ГОСТ 126-79\xa0Галоши резиновые клееные. Технические условия.txt',
 'ГОСТ 18165-89\xa0Вода питьевая. Метод определения массовой концентрации алюминия.txt',
 'ГОСТ 18321-73\xa0Статистический контроль качества. Методы случайного отбора выборок штучной продукции.txt',
 'ГОСТ 24295-80 Посуда хозяйственная стальная эмалированная. Методы анализа вытяжек.txt',
 'ГОСТ 24788-2001\xa0Посуда хозяйственная стальная эмалированная. Общие технические условия.txt',
 'ГОСТ 31870-2012\xa0Вода питьевая. Определение содержания элементов методами атомной спектрометрии.txt',
 'ГОСТ 31949-2012\xa0Вода питьевая. Метод определения содержания бора.txt',
 'ГОСТ 32075-2013\xa0Материалы текстильные. Метод определения токсичности.txt',
 'ГОСТ 413-91 (ИСО 1420-87) Ткани с резиновым или пластмассовым покрытием. Определение....txt',
 'ГОСТ 4152-89\xa0Вода питьевая. Метод определения массовой концентрации мышьяка.txt',
 'ГОСТ 4386-89\xa0Вода питьевая. Методы определения массовой концентрации фторидов.txt',
 'ГОСТ 9.3

# Изучаем Датасет

1. Файл **dataset.csv** содержит информацию о связи стандарта-продукции и технического оборудования и следующие поля:
- id - уникальный идентификатор продукции
- Обозначение стандарта - соответствие продукции стандартам
- Группа продукции 
- Наименование продукции
- Коды ОКПД 2 / ТН ВЭД ЕАЭС - идентификаторы продукции
- Техническое оборудование - список оборудования для проверки продукции соответствия стандартам.

In [4]:
data

,id,Обозначение стандарта,Группа продукции,Наименование продукции,Коды ОКПД 2 / ТН ВЭД ЕАЭС,Техническое оборудование
0,1,"ГОСТ 3897-2015, ГОСТ 8541-2014","Колготки, носки, получулки, чулки 1-го слоя и ...","ИЗДЕЛИЯ ЧУЛОЧНО-НОСОЧНЫЕ первого слоя, вырабат...",6115290000; 6115950000,маркировочное устройство; тарированный спидоме...
1,2,"ГОСТ 31410-2009 , ГОСТ 31409-2009","Свитеры, джемперы, жакеты, юбки, брюки, костюм...",Изделия верхние трикотажные второго слоя для д...,6110; 6106; 6104; 6103,Хроматографическая колонка; капилляры; Весы ан...
2,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...
3,4,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская (малодетская, дошкольная, для шк...",6404199000,Хроматографическая колонка; капилляры; Весы ан...
4,5,"ГОСТ 29235-91 (ИСО 6742-2-85), ГОСТ 28765-90 (...",Велосипеды с высотой седла от 435 мм до 635 мм...,Велосипеды двухколесные из металла с элементам...,8712003000,маркировочное устройство; тарированный спидоме...
...,...,...,...,...,...,...
12820,12821,"ГОСТ 3897-2015, ГОСТ 31408-2009","Пижамы, кальсоны, панталоны, фуфайки, комбинац...",ИЗДЕЛИЯ ПЕРВОГО СЛОЯ ТРИКОТАЖНЫЕ БЕЛЬЕВЫЕ для ...,6107210000; 6109100000,Хроматографическая колонка; капилляры; Весы ан...
12821,12822,"ГОСТ 30327-2013, ГОСТ 25295-2003, ГОСТ 25294-2003","Платья, халаты, фартуки, блузки, верхние сороч...",Одежда верхняя швейная второго слоя для детей ...,6203120000; 6203191000; 6203329000; 6203339000...,Хроматографическая колонка; капилляры; Весы ан...
12822,12823,ГОСТ 5274-2014,"Перчатки, варежки, платки, шарфы и аналогичные...",Изделия трикотажные платочно-шарфового ассорти...,6111209000; 6111309000; 6117100000,U-образный стеклянный капилляр; Аналитические ...
12823,12824,"ГОСТ 3897-2015, ГОСТ 31410-2009","Свитеры, джемперы, жакеты, юбки, брюки, костюм...",ИЗДЕЛИЯ ТРИКОТАЖНЫЕ ВЕРХНИЕ ВТОРОГО СЛОЯ для м...,6110209100,Хроматографическая колонка; капилляры; Весы ан...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12825 entries, 0 to 12824
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         12825 non-null  int64 
 1   Обозначение стандарта      12825 non-null  object
 2   Группа продукции           12825 non-null  object
 3   Наименование продукции     12825 non-null  object
 4   Коды ОКПД 2 / ТН ВЭД ЕАЭС  12825 non-null  object
 5   Техническое оборудование   12825 non-null  object
dtypes: int64(1), object(5)
memory usage: 601.3+ KB


In [6]:
unique_values = data['Группа продукции'].unique()
print(unique_values)

['Колготки, носки, получулки, чулки 1-го слоя и аналогичные изделия, заявленные изготовителем как предназначенные для детей старше 3 лет и подростков'
 'Свитеры, джемперы, жакеты, юбки, брюки, костюмы, рейтузы, полукомбинезоны, комбинезоны, шорты, платья, комплекты, блузки, сорочки верхние, жилеты и аналогичные изделия, заявленные изготовителем как предназначенные для детей и подростков'
 'Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков'
 'Велосипеды с высотой седла от 435 мм до 635 мм для детей дошкольного возраста, транспортные велосипеды с регулировкой седла на высоту 635 мм и более для младших школьников и подростков'
 'Галантерейные изделия детские из пластмасс'
 'Платья, халаты, фартуки, блузки, верхние сорочки, свитеры, джемперы, шорты, изделия без 

## stds_ls

Файл **standarts_list.csv** - список фигурирующих в кейсе стандартов, тексты будут выданы непосредственно во время проведения хакатона в электронном виде.

In [7]:
stds_ls

,Обозначение и наименование документов в области стандартизации,Группа продукции
0,"МУ 1.1.037-95 ""Биотестирование продукции из по...","изделия из пластмасс (ванночка, горшок туалетн..."
1,"ГОСТ 18165-89 ""Вода питьевая. Метод определени...","Посуда и столовые приборы (чашки, блюдца, поил..."
2,"ГОСТ 18321-73 ""Статистический контроль качеств...","изделия из пластмасс (ванночка, горшок туалетн..."
3,"РД 52.04.186-89 ""Руководство по контролю загря...","Детская литература, журналы и продолжающиеся и..."
4,"ПНД Ф 14.1:2:4.140-98 ""Методика выполнения изм...","Посуда и столовые приборы (чашки, блюдца, поил..."
5,"ПНД Ф 14.1:2:4.140-98 ""Методика выполнения изм...","Детская литература, журналы и продолжающиеся и..."
6,"ГОСТ 31870-2012 ""Вода питьевая. Определение со...","Посуда и столовые приборы (чашки, блюдца, поил..."
7,"ПНД Ф 14.1:2:4.139-98 ""Методика выполнения изм...","Посуда и столовые приборы (чашки, блюдца, поил..."
8,"ПНД Ф 14.1:2:4.143-98 ""Методика выполнения изм...","Посуда и столовые приборы (чашки, блюдца, поил..."
9,"МУК 4.1.617-96 ""Методические указания по газох...","Детская литература, журналы и продолжающиеся и..."


In [8]:
stds_ls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 2 columns):
 #   Column                                                          Non-Null Count  Dtype 
---  ------                                                          --------------  ----- 
 0   Обозначение и наименование документов в области стандартизации  58 non-null     object
 1   Группа продукции                                                58 non-null     object
dtypes: object(2)
memory usage: 1.0+ KB


In [9]:
unique_values = stds_ls['Группа продукции'].unique()
print(unique_values)

['изделия из пластмасс (ванночка, горшок туалетный и другие изделия для выполнения туалета) для ухода за детьми'
 'Посуда и столовые приборы (чашки, блюдца, поильники, тарелки, миски, ложки, вилки, ножи, бутылочки и другие аналогичные изделия детские для пищевых продуктов) из пластмассы, стекла, металла, посуда керамическая (фаянсовая, стеклокерамическая, гончарная и майоликовая), посуда одноразовая (из бумаги, картона и пластмассы), заявленные изготовителем как предназначенные для детей старше 3 лет и подростков'
 'Детская литература, журналы и продолжающиеся издания детские'
 'Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков'
 'Коляски детские, комплектующие узлы и детали к ним']


## stds

In [10]:
stds

,N п/п из Перечня,Группа продукции,Структурный элемент или объект,Пункт/статья в ТР ТС,Пункт/раздел стандарта,Обозначение и наименование стандарта
0,1,Изделия для ухода за детьми,"токсикологические показатели, индекс токсичности",статья 4,NaN,"МУ 1.1.037-95 ""Биотестирование продукции из по..."
1,2,Изделия для ухода за детьми,"токсикологические показатели, индекс токсичности",статья 4,NaN,"МР N 29 ФЦ/2688-03 ""Экспресс-метод оценки токс..."
2,3,Изделия для ухода за детьми,"токсикологические показатели, индекс токсичности",статья 4,NaN,"ГОСТ ISO 10993-10-2011 ""Изделия медицинские. О..."
3,4,Изделия для ухода за детьми,"токсикологические показатели, индекс токсичности",статья 4,NaN,"Инструкция N 1.1.11-12-35-2004 ""Требования к п..."
4,5,Изделия для ухода за детьми,"токсикологические показатели, индекс токсичности",статья 4,NaN,"ГОСТ 32893-2014 ""Продукция парфюмерно-косметич..."
...,...,...,...,...,...,...
1487,1486,Школьно-письменные принадлежности,"масса бумаги площадью 1м2, толщина линий, обра...",статья 8,NaN,"ГОСТ 13199-94 ""Полуфабрикаты волокнистые, бума..."
1488,1487,Школьно-письменные принадлежности,"масса бумаги площадью 1м2, толщина линий, обра...",статья 8,пункты 3.4 и 3.5,"ГОСТ 13309-90 ""Тетради общие. Технические усло..."
1489,1488,Школьно-письменные принадлежности,"масса бумаги площадью 1м2, толщина линий, обра...",статья 8,пункты 3.4 и 3.5,"ГОСТ 12063-89 ""Тетради школьные. Технические у..."
1490,1489,Школьно-письменные принадлежности,"масса бумаги площадью 1м2, толщина линий, обра...",статья 8,пункты 9.6 и 9.13,"ГОСТ Р 54543-2011 ""Тетради ученические. Общие ..."


In [11]:
unique_values = stds['Группа продукции'].unique()
print(unique_values)

['Изделия для ухода за детьми'
 'Соски молочные, соски-пустышки и изделия санитарно-гигиенические из латекса, резины и силиконовых эластомеров'
 'Посуда и столовые приборы из пластмассы'
 'Посуда и столовые приборы из стекла, стеклокерамики, керамики'
 'Посуда и столовые приборы из металла, изделия санитарно-гигиенические из металла'
 'Посуда из бумаги и картона (одноразового применения)'
 'Изделия санитарно-гигиенические и галантерейные из пластмассы'
 'Щетки зубные, массажеры для десен и аналогичные изделия, предназначенные для ухода за полостью рта'
 'Изделия санитарно-гигиенические разового использования'
 'Одежда, изделия из текстильных материалов и меха, трикотажные изделия, готовые штучные текстильные изделия и текстильные материалы, используемые для изготовления обуви, одежды и изделий из кожи, кожгалантерейных изделий и колясок'
 'Одежда и изделия из текстильных материалов, меха, изделия трикотажные, готовые штучные текстильные изделия, обувь, кожгалантерейные изделия, коляски

# Обрабатываем данные

Соединяем таблички по группе продукции

In [12]:
merged_df = data.merge(stds_ls, on='Группа продукции', how='left')

In [13]:
merged_df

,id,Обозначение стандарта,Группа продукции,Наименование продукции,Коды ОКПД 2 / ТН ВЭД ЕАЭС,Техническое оборудование,Обозначение и наименование документов в области стандартизации
0,1,"ГОСТ 3897-2015, ГОСТ 8541-2014","Колготки, носки, получулки, чулки 1-го слоя и ...","ИЗДЕЛИЯ ЧУЛОЧНО-НОСОЧНЫЕ первого слоя, вырабат...",6115290000; 6115950000,маркировочное устройство; тарированный спидоме...,NaN
1,2,"ГОСТ 31410-2009 , ГОСТ 31409-2009","Свитеры, джемперы, жакеты, юбки, брюки, костюм...",Изделия верхние трикотажные второго слоя для д...,6110; 6106; 6104; 6103,Хроматографическая колонка; капилляры; Весы ан...,NaN
2,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ Р ИСО 16000-6-2007 ""Воздух замкнутых поме..."
3,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ 9135-2004 ""Обувь. Метод определения общей..."
4,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ 9718-88 ""Обувь. Метод определения гибкости"""
...,...,...,...,...,...,...,...
36337,12821,"ГОСТ 3897-2015, ГОСТ 31408-2009","Пижамы, кальсоны, панталоны, фуфайки, комбинац...",ИЗДЕЛИЯ ПЕРВОГО СЛОЯ ТРИКОТАЖНЫЕ БЕЛЬЕВЫЕ для ...,6107210000; 6109100000,Хроматографическая колонка; капилляры; Весы ан...,NaN
36338,12822,"ГОСТ 30327-2013, ГОСТ 25295-2003, ГОСТ 25294-2003","Платья, халаты, фартуки, блузки, верхние сороч...",Одежда верхняя швейная второго слоя для детей ...,6203120000; 6203191000; 6203329000; 6203339000...,Хроматографическая колонка; капилляры; Весы ан...,NaN
36339,12823,ГОСТ 5274-2014,"Перчатки, варежки, платки, шарфы и аналогичные...",Изделия трикотажные платочно-шарфового ассорти...,6111209000; 6111309000; 6117100000,U-образный стеклянный капилляр; Аналитические ...,NaN
36340,12824,"ГОСТ 3897-2015, ГОСТ 31410-2009","Свитеры, джемперы, жакеты, юбки, брюки, костюм...",ИЗДЕЛИЯ ТРИКОТАЖНЫЕ ВЕРХНИЕ ВТОРОГО СЛОЯ для м...,6110209100,Хроматографическая колонка; капилляры; Весы ан...,NaN


In [14]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36342 entries, 0 to 36341
Data columns (total 7 columns):
 #   Column                                                          Non-Null Count  Dtype 
---  ------                                                          --------------  ----- 
 0   id                                                              36342 non-null  int64 
 1   Обозначение стандарта                                           36342 non-null  object
 2   Группа продукции                                                36342 non-null  object
 3   Наименование продукции                                          36342 non-null  object
 4   Коды ОКПД 2 / ТН ВЭД ЕАЭС                                       36342 non-null  object
 5   Техническое оборудование                                        36342 non-null  object
 6   Обозначение и наименование документов в области стандартизации  26194 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.9+ MB


In [15]:
unique_values = merged_df['Обозначение и наименование документов в области стандартизации'].unique()
print(unique_values)

[nan
 'ГОСТ Р ИСО 16000-6-2007\xa0"Воздух замкнутых помещений. Часть 6. Определение летучих органических соединений в воздухе замкнутых помещений и испытательной камеры путем активного отбора проб на сорбент Теnах ТА с последующей термической десорбцией и газохроматографическим анализом с использованием МСД/ПИД'
 'ГОСТ 9135-2004\xa0"Обувь. Метод определения общей и остаточной деформации подноска и задника"'
 'ГОСТ 9718-88\xa0"Обувь. Метод определения гибкости"'
 'ГОСТ Р ИСО 17131-2014\xa0"Кожа. Метод идентификации с помощью микроскопа"'
 'СТ РК ИСО 20864-2011\xa0"Обувь. Метод испытаний жестких задников и подносков. Механические характеристики"'
 'ГОСТ 9134-78\xa0"Обувь. Метод определения прочности крепления деталей низа"'
 'ГОСТ 9292-82\xa0"Обувь. Метод определения прочности крепления подошв в обуви химических методов крепления"'
 'п. 4.9\xa0ГОСТ 126-79\xa0"Галоши резиновые клееные. Технические условия"'
 'ГОСТ Р 53485-2009\xa0"Материалы текстильные. Метод определения токсичности"'
 'Г

Удаляем строки, для которых не нашлось стандартов

In [16]:
merged_df = merged_df.dropna()

Объявляем фунцкию на основе fuzzywuzzy

In [17]:
def predict(gost_name, threshold=60):
    files_list = files
    matches = process.extractOne(gost_name, files_list, scorer=fuzz.partial_ratio)
    return matches[0] if matches[1] > threshold else None

Прогоняем через функцию все документы из строк

In [ ]:
merged_df['Файл стандарта'] = ''

for i in range(len(merged_df)):
    value = merged_df['Обозначение и наименование документов в области стандартизации'].iloc[i]
    result = predict(value)
    merged_df['Файл стандарта'].iloc[i] = result

In [19]:
merged_df

,id,Обозначение стандарта,Группа продукции,Наименование продукции,Коды ОКПД 2 / ТН ВЭД ЕАЭС,Техническое оборудование,Обозначение и наименование документов в области стандартизации,Файл стандарта
2,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ Р ИСО 16000-6-2007 ""Воздух замкнутых поме...",ГОСТ Р ИСО 16000-6-2007 Воздух замкнутых помещ...
3,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ 9135-2004 ""Обувь. Метод определения общей...",ГОСТ 9135-2004 Обувь. Метод определения общей ...
4,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ 9718-88 ""Обувь. Метод определения гибкости""",ГОСТ 9718-88 Обувь. Метод определения гибкости...
5,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ Р ИСО 17131-2014 ""Кожа. Метод идентификац...",ГОСТ Р ИСО 17131-2014 Кожа. Метод идентификаци...
6,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,"СТ РК ИСО 20864-2011 ""Обувь. Метод испытаний ж...",СТ РК ИСО 20864-2011 Обувь. Методы испытаний ж...
...,...,...,...,...,...,...,...,...
36320,12808,"ГОСТ 30804.6.1-2013 (IEC 61000-6-1:2005), ГОСТ...","Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская и подростковая (дошкольная, для ...",6402999100,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ 9134-78 ""Обувь. Метод определения прочнос...",ГОСТ 9134-78 Обувь. Методы определения прочнос...
36321,12808,"ГОСТ 30804.6.1-2013 (IEC 61000-6-1:2005), ГОСТ...","Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская и подростковая (дошкольная, для ...",6402999100,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ 9292-82 ""Обувь. Метод определения прочнос...",ГОСТ 9292-82 Обувь. Метод определения прочност...
36322,12808,"ГОСТ 30804.6.1-2013 (IEC 61000-6-1:2005), ГОСТ...","Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская и подростковая (дошкольная, для ...",6402999100,Хроматографическая колонка; капилляры; Весы ан...,"п. 4.9 ГОСТ 126-79 ""Галоши резиновые клееные. ...",ГОСТ 126-79 Галоши резиновые клееные. Техничес...
36323,12808,"ГОСТ 30804.6.1-2013 (IEC 61000-6-1:2005), ГОСТ...","Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская и подростковая (дошкольная, для ...",6402999100,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ Р 53485-2009 ""Материалы текстильные. Мето...",ГОСТ Р 53485-2009 Материалы текстильные. Метод...


In [26]:
print(merged_df.columns)

Index(['id', 'Обозначение стандарта', 'Группа продукции',
       'Наименование продукции', 'Коды ОКПД 2 / ТН ВЭД ЕАЭС',
       'Техническое оборудование',
       'Обозначение и наименование документов в области стандартизации',
       'Файл стандарта', 'Должно быть по факту'],
      dtype='object')


In [27]:
#print(gdf.iloc[0].values)

Составляем базу уникальных значений оборудования из всех стандартов

In [28]:
unique_equipment = set()

for index, row in merged_df.iterrows():
    equipment_list = row['Техническое оборудование'].split('; ')
    unique_equipment.update(equipment_list)

unique_equipment_list = list(unique_equipment)

print(unique_equipment_list)

['Посуда стеклянная из темного и светлого стекла для хранения проб', 'Фильтр "белая лента"', 'кюветы', 'электротермический атомизатор', 'Груша', 'оборудование для испытания на тепловой удар', 'мерная колба', 'флуориметры', 'Фотометр', 'устройство для обработки выходных сигналов спектрометра с возможностью коррекции фоновых сигналов', 'редуктор кислородный', 'Градуировочная характеристика', 'Редуктор балонный ДКП-1-65', 'Графитовая печь', 'электрическая плитка для нагрева раствора уксусной кислоты', 'термостатическая ванна', 'Спектрометр Analyst 600', 'это атомно-абсорбционный спектрометр', 'Колбы мерные вместимостью 25', 'фильтр "красная лента"', 'весы лабораторные обычного класса точности', 'флуориметрический метод измерения массовой концентрации цинка в пробах воды', 'аспиратор', 'Устройство ввода газового хроматографа', 'светофильтры', 'Фильтры обеззоленные синяя лента ТУ 6-09-1678-86', 'Делительная воронка', 'Цилиндр мерный', 'сорбционная трубка', 'Электроаспиратор', 'Весы аналитич

Идем по изначальным спискам оборудования, лемматизируем, сравниваем с тем, что есть в документе

In [29]:
nlp = spacy.load('ru_core_news_sm')

def lemmatize_term(term):
    doc = nlp(term)
    lemmatized_term = " ".join(token.lemma_ for token in doc)
    return lemmatized_term

def find_terms_in_text(file_path, search_terms, text):
    if file_path is not None:
        file_path = 'files/' + file_path
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
    
    lemmatized_text = " ".join(token.lemma_ for token in nlp(text))
    
    found_terms = [term for term in search_terms if lemmatize_term(term).lower() in lemmatized_text.lower()]
    
#     if len(found_terms) == len(search_terms):
#         print("Все термины найдены в тексте.")
#         print("Найденные термины:", found_terms)
#     else:
#         print(f"Файл: {file_path} \n")
#         print("Не все термины найдены в тексте.")
#         print(f"Что было в таблице: {search_terms} \n")
#         print(f"Что нашли: {found_terms}\n")
#         print(f"Должно быть по факту: {find_correct(file_path)}")
    
    return 'Верно' if len(found_terms) == len(search_terms) else 'Неверно'

def find_correct(file_path):
    file_path = 'files/' + file_path
    search_terms = unique_equipment
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()

    lemmatized_text = " ".join(token.lemma_ for token in nlp(text))
    found_terms = [term for term in search_terms if lemmatize_term(term).lower() in lemmatized_text.lower()]
    
    return found_terms


In [30]:
merged_df['Должно быть по факту'] = ''

# for i in range(len(merged_df)):
    
for i in range(0,20):
    print(f"Проверяю {merged_df['Группа продукции'].iloc[i]}")
    search_terms = (merged_df['Техническое оборудование'].iloc[i]).split('; ')
    result = find_terms_in_text(None, search_terms, merged_df['Файл стандарта'].iloc[i])
    if result != "Верно":
        merged_df['Должно быть по факту'].iloc[i] = '; '.join(find_correct(merged_df['Файл стандарта'].iloc[i]))
    #merged_df['Прошло проверку'].iloc[i] = result

C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\2486282084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Должно быть по факту'] = ''


Проверяю Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков


C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\2486282084.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Должно быть по факту'].iloc[i] = '; '.join(find_correct(merged_df['Файл стандарта'].iloc[i]))


Проверяю Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков


C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\2486282084.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Должно быть по факту'].iloc[i] = '; '.join(find_correct(merged_df['Файл стандарта'].iloc[i]))


...

C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\2486282084.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Должно быть по факту'].iloc[i] = '; '.join(find_correct(merged_df['Файл стандарта'].iloc[i]))


Проверяю Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков


C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\2486282084.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Должно быть по факту'].iloc[i] = '; '.join(find_correct(merged_df['Файл стандарта'].iloc[i]))


Проверяю Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков


C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\2486282084.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['Должно быть по факту'].iloc[i] = '; '.join(find_correct(merged_df['Файл стандарта'].iloc[i]))


In [31]:
merged_df

,id,Обозначение стандарта,Группа продукции,Наименование продукции,Коды ОКПД 2 / ТН ВЭД ЕАЭС,Техническое оборудование,Обозначение и наименование документов в области стандартизации,Файл стандарта,Должно быть по факту
2,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ Р ИСО 16000-6-2007 ""Воздух замкнутых поме...",ГОСТ Р ИСО 16000-6-2007 Воздух замкнутых помещ...,мерная колба; Градуировочная характеристика; У...
3,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ 9135-2004 ""Обувь. Метод определения общей...",ГОСТ 9135-2004 Обувь. Метод определения общей ...,Набор съемных грузов; Набор съемных наконечник...
4,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ 9718-88 ""Обувь. Метод определения гибкости""",ГОСТ 9718-88 Обувь. Метод определения гибкости...,весы; Разрывная машина; устройство для креплен...
5,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ Р ИСО 17131-2014 ""Кожа. Метод идентификац...",ГОСТ Р ИСО 17131-2014 Кожа. Метод идентификаци...,весы; Оптический микроскоп; Растровый электрон...
6,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,"СТ РК ИСО 20864-2011 ""Обувь. Метод испытаний ж...",СТ РК ИСО 20864-2011 Обувь. Методы испытаний ж...,Шпиндель толщиномера; весы; Комплект испытател...
...,...,...,...,...,...,...,...,...,...
36320,12808,"ГОСТ 30804.6.1-2013 (IEC 61000-6-1:2005), ГОСТ...","Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская и подростковая (дошкольная, для ...",6402999100,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ 9134-78 ""Обувь. Метод определения прочнос...",ГОСТ 9134-78 Обувь. Методы определения прочнос...,
36321,12808,"ГОСТ 30804.6.1-2013 (IEC 61000-6-1:2005), ГОСТ...","Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская и подростковая (дошкольная, для ...",6402999100,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ 9292-82 ""Обувь. Метод определения прочнос...",ГОСТ 9292-82 Обувь. Метод определения прочност...,
36322,12808,"ГОСТ 30804.6.1-2013 (IEC 61000-6-1:2005), ГОСТ...","Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская и подростковая (дошкольная, для ...",6402999100,Хроматографическая колонка; капилляры; Весы ан...,"п. 4.9 ГОСТ 126-79 ""Галоши резиновые клееные. ...",ГОСТ 126-79 Галоши резиновые клееные. Техничес...,
36323,12808,"ГОСТ 30804.6.1-2013 (IEC 61000-6-1:2005), ГОСТ...","Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская и подростковая (дошкольная, для ...",6402999100,Хроматографическая колонка; капилляры; Весы ан...,"ГОСТ Р 53485-2009 ""Материалы текстильные. Мето...",ГОСТ Р 53485-2009 Материалы текстильные. Метод...,


In [32]:
gdf = merged_df.groupby(['id', 'Обозначение стандарта', 'Группа продукции',
       'Наименование продукции', 'Коды ОКПД 2 / ТН ВЭД ЕАЭС',
       'Техническое оборудование'])['Должно быть по факту'].agg(list).reset_index()

In [33]:
gdf

,id,Обозначение стандарта,Группа продукции,Наименование продукции,Коды ОКПД 2 / ТН ВЭД ЕАЭС,Техническое оборудование,Должно быть по факту
0,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,[мерная колба; Градуировочная характеристика; ...
1,4,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская (малодетская, дошкольная, для шк...",6404199000,Хроматографическая колонка; капилляры; Весы ан...,[мерная колба; Градуировочная характеристика; ...
2,22,"ГОСТ Р 51687-2000, ГОСТ Р 50962-96","Посуда и столовые приборы (чашки, блюдца, поил...",Приборы столовые для детей страше 3-х лет в на...,8215201000,pH-метр любой модели; Весы лабораторные по ГОС...,"[, , , , , , , , , , , , , , , , , , , , , , ]"
3,25,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская (для школьников-мальчиков, мальч...",6402999600; 6402999100; 6402919000,Хроматографическая колонка; капилляры; Весы ан...,"[, , , , , , , , , ]"
4,27,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская (дошкольная, для школьников-дево...",6402999100,Хроматографическая колонка; капилляры; Весы ан...,"[, , , , , , , , , ]"
...,...,...,...,...,...,...,...
2672,12784,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская повседневная (ясельная, малодетс...",6405100009,Хроматографическая колонка; капилляры; Весы ан...,"[, , , , , , , , , ]"
2673,12788,ГОСТ 1135-2005,"Сапоги, сапожки, полусапожки, ботинки, полубот...",Обувь домашняя для детей (школьников-мальчиков...,6405209100,Хроматографическая колонка; капилляры; Весы ан...,"[, , , , , , , , , ]"
2674,12804,Техническое решение и оценка рисков,"Сапоги, сапожки, полусапожки, ботинки, полубот...",Обувь повседневная для детей (для дошкольников...,6404,Хроматографическая колонка; капилляры; Весы ан...,"[, , , , , , , , , ]"
2675,12805,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь повседневная для детей малодетской, дошк...",6404199000,Хроматографическая колонка; капилляры; Весы ан...,"[, , , , , , , , , ]"


In [34]:
print(gdf.iloc[0].values)

[3 'ГОСТ 26165-2003'
 'Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков'
 'Обувь детская летняя (дошкольная, для школьников-девочек) с верхом из искусственной кожи, с элементами отделки из текстильных материалов, с подошвой из резины, клеевого метода крепления:'
 '6402993900'
 'Хроматографическая колонка; капилляры; Весы аналитические с погрешностью взвешивания не более 1 мг; Дозаторы пипеточные на объемы 0,5 мл; Комплект кювет; Мерные колбы с притертыми пробками объемом 50 мл; Пинцет анатомический длиной 250 мм; Пробирки с притертыми пробками объемом 3-5 мл; Сосуд Дьюара типа СДС объемом не менее 25 л и диаметром горла не менее 50 мм; анатомический пинцет; блок подготовки проб и рабочих образцов; блок термостатирования; водный термостат; замороженная в жид

In [35]:
gdf['Проверка'] = ''

# for i in range(len(merged_df)):
    
for i in range(0,20):
    print(f"Проверяю {gdf['Группа продукции'].iloc[i]}")
    search_terms = (gdf['Должно быть по факту'].iloc[i])
    result = find_terms_in_text(None, search_terms, gdf['Техническое оборудование'].iloc[i])
    gdf['Проверка'].iloc[i] = result

Проверяю Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков


C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\757481081.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['Проверка'].iloc[i] = result


Проверяю Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков


C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\757481081.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['Проверка'].iloc[i] = result


Проверяю Посуда и столовые приборы (чашки, блюдца, поильники, тарелки, миски, ложки, вилки, ножи, бутылочки и другие аналогичные изделия детские для пищевых продуктов) из пластмассы, стекла, металла, посуда керамическая (фаянсовая, стеклокерамическая, гончарная и майоликовая), посуда одноразовая (из бумаги, картона и пластмассы), заявленные изготовителем как предназначенные для детей старше 3 лет и подростков


...

C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\757481081.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['Проверка'].iloc[i] = result
C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\757481081.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['Проверка'].iloc[i] = result


Проверяю Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков
Проверяю Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков


C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\757481081.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['Проверка'].iloc[i] = result
C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\757481081.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['Проверка'].iloc[i] = result


Проверяю Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков
Проверяю Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков


C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\757481081.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdf['Проверка'].iloc[i] = result


In [36]:
gdf

,id,Обозначение стандарта,Группа продукции,Наименование продукции,Коды ОКПД 2 / ТН ВЭД ЕАЭС,Техническое оборудование,Должно быть по факту,Проверка
0,3,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская летняя (дошкольная, для школьник...",6402993900,Хроматографическая колонка; капилляры; Весы ан...,[мерная колба; Градуировочная характеристика; ...,Неверно
1,4,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская (малодетская, дошкольная, для шк...",6404199000,Хроматографическая колонка; капилляры; Весы ан...,[мерная колба; Градуировочная характеристика; ...,Неверно
2,22,"ГОСТ Р 51687-2000, ГОСТ Р 50962-96","Посуда и столовые приборы (чашки, блюдца, поил...",Приборы столовые для детей страше 3-х лет в на...,8215201000,pH-метр любой модели; Весы лабораторные по ГОС...,"[, , , , , , , , , , , , , , , , , , , , , , ]",Верно
3,25,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская (для школьников-мальчиков, мальч...",6402999600; 6402999100; 6402919000,Хроматографическая колонка; капилляры; Весы ан...,"[, , , , , , , , , ]",Верно
4,27,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская (дошкольная, для школьников-дево...",6402999100,Хроматографическая колонка; капилляры; Весы ан...,"[, , , , , , , , , ]",Верно
...,...,...,...,...,...,...,...,...
2672,12784,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь детская повседневная (ясельная, малодетс...",6405100009,Хроматографическая колонка; капилляры; Весы ан...,"[, , , , , , , , , ]",
2673,12788,ГОСТ 1135-2005,"Сапоги, сапожки, полусапожки, ботинки, полубот...",Обувь домашняя для детей (школьников-мальчиков...,6405209100,Хроматографическая колонка; капилляры; Весы ан...,"[, , , , , , , , , ]",
2674,12804,Техническое решение и оценка рисков,"Сапоги, сапожки, полусапожки, ботинки, полубот...",Обувь повседневная для детей (для дошкольников...,6404,Хроматографическая колонка; капилляры; Весы ан...,"[, , , , , , , , , ]",
2675,12805,ГОСТ 26165-2003,"Сапоги, сапожки, полусапожки, ботинки, полубот...","Обувь повседневная для детей малодетской, дошк...",6404199000,Хроматографическая колонка; капилляры; Весы ан...,"[, , , , , , , , , ]",


In [ ]:
for i in range(len(gdf)):
    gdf['Должно быть по факту'].iloc[i] = '; '.join(gdf['Должно быть по факту'].iloc[i])

In [38]:
print(gdf.iloc[0].values)

[3 'ГОСТ 26165-2003'
 'Сапоги, сапожки, полусапожки, ботинки, полуботинки, туфли, сандалеты и другие виды обуви из юфти, хромовых кож, текстильных синтетических и искусственных материалов, резиновые, резинотекстильные и комбинированные, заявленные изготовителем как предназначенные для детей и подростков'
 'Обувь детская летняя (дошкольная, для школьников-девочек) с верхом из искусственной кожи, с элементами отделки из текстильных материалов, с подошвой из резины, клеевого метода крепления:'
 '6402993900'
 'Хроматографическая колонка; капилляры; Весы аналитические с погрешностью взвешивания не более 1 мг; Дозаторы пипеточные на объемы 0,5 мл; Комплект кювет; Мерные колбы с притертыми пробками объемом 50 мл; Пинцет анатомический длиной 250 мм; Пробирки с притертыми пробками объемом 3-5 мл; Сосуд Дьюара типа СДС объемом не менее 25 л и диаметром горла не менее 50 мм; анатомический пинцет; блок подготовки проб и рабочих образцов; блок термостатирования; водный термостат; замороженная в жид

# Прогоняем тестовые данные

In [82]:
test = pd.read_csv("Test - Team NAME.csv")

In [83]:
test

,id,Обозначение стандарта,Группа продукции,Наименование продукции,Коды ОКПД 2 / ТН ВЭД ЕАЭС,Техническое оборудование,Укажите верно ли указано техническое оборудование?,Ваш вариант технического оборудования
0,1,"ГОСТ 10581-91, ГОСТ 32119-2013",Изделия на подкладке или без подкладки (пальто...,Одежда 3-го слоя для детей до одного года с ве...,6209909000,U-образный стеклянный капилляр; Аналитические ...,,NaN
1,2,ГОСТ 19245-93,"Коляски детские, комплектующие узлы и детали к...",Коляски детские комбинированные с открытыми и ...,8715009000; 8715001000,Прибор для испытания устойчивости окраски к тр...,,NaN
2,3,"ГОСТ 25294-2003, ГОСТ 10581-91, ГОСТ 30386-95,...",Изделия на подкладке или без подкладки (пальто...,Изделия швейные (3-ий слой) для эпизодической ...,6204,U-образный стеклянный капилляр; Аналитические ...,,NaN
3,4,ГОСТ 25295-2003,Изделия на подкладке или без подкладки (пальто...,Одежда 3-го слоя верхняя швейная пальтового ас...,6209300000,U-образный стеклянный капилляр; Аналитические ...,,NaN
4,5,"ГОСТ 25295-2003, ГОСТ 25295-2003, ГОСТ 25295-2...",Изделия на подкладке или без подкладки (пальто...,Изделия 2-го слоя верхние швейные костюмного а...,6203423500; 6204623900; 6204629000,U-образный стеклянный капилляр; Аналитические ...,,NaN
5,6,"ГОСТ 29235-91 (ИСО 6742-2-85), ГОСТ 28765-90 (...",Велосипеды с высотой седла от 435 мм до 635 мм...,Велосипеды двухколесные из металла с элементам...,8712003000,маркировочное устройство; тарированный спидоме...,,NaN
6,7,ГОСТ 30407-96,"Посуда и столовые приборы (чашки, блюдца, поил...",Посуда из стеклокерамики,7013100000,pH-метр любой модели; Весы лабораторные по ГОС...,,NaN
7,8,ГОСТ 31409-2009,"Пальто, куртки и аналогичные изделия, заявленн...",Изделия верхние трикотажные третьего слоя для ...,6102209000,U-образный стеклянный капилляр; Аналитические ...,,NaN
8,9,"ГОСТ 31410-2009 , ГОСТ 31409-2009","Свитеры, джемперы, жакеты, юбки, брюки, костюм...",Изделия верхние трикотажные второго слоя для д...,6110; 6106; 6104; 6103,Хроматографическая колонка; капилляры; Весы ан...,,NaN
9,10,ГОСТ 32118-2013,Головные уборы (летние) 1-го слоя и аналогичны...,Головные уборы (летние) 1-го слоя,6505009000,U-образный стеклянный капилляр; Аналитические ...,,NaN


In [84]:
m_test = test.merge(stds_ls, on='Группа продукции', how='left')

In [85]:
m_test = m_test.fillna("Нет данных")

In [86]:
m_test

,id,Обозначение стандарта,Группа продукции,Наименование продукции,Коды ОКПД 2 / ТН ВЭД ЕАЭС,Техническое оборудование,Укажите верно ли указано техническое оборудование?,Ваш вариант технического оборудования,Обозначение и наименование документов в области стандартизации
0,1,"ГОСТ 10581-91, ГОСТ 32119-2013",Изделия на подкладке или без подкладки (пальто...,Одежда 3-го слоя для детей до одного года с ве...,6209909000,U-образный стеклянный капилляр; Аналитические ...,,Нет данных,Нет данных
1,2,ГОСТ 19245-93,"Коляски детские, комплектующие узлы и детали к...",Коляски детские комбинированные с открытыми и ...,8715009000; 8715001000,Прибор для испытания устойчивости окраски к тр...,,Нет данных,"ГОСТ 9733.27-83 ""Материалы текстильные. Метод ..."
2,2,ГОСТ 19245-93,"Коляски детские, комплектующие узлы и детали к...",Коляски детские комбинированные с открытыми и ...,8715009000; 8715001000,Прибор для испытания устойчивости окраски к тр...,,Нет данных,"ГОСТ 413-91 ""Ткани с резиновым или пластмассов..."
3,3,"ГОСТ 25294-2003, ГОСТ 10581-91, ГОСТ 30386-95,...",Изделия на подкладке или без подкладки (пальто...,Изделия швейные (3-ий слой) для эпизодической ...,6204,U-образный стеклянный капилляр; Аналитические ...,,Нет данных,Нет данных
4,4,ГОСТ 25295-2003,Изделия на подкладке или без подкладки (пальто...,Одежда 3-го слоя верхняя швейная пальтового ас...,6209300000,U-образный стеклянный капилляр; Аналитические ...,,Нет данных,Нет данных
...,...,...,...,...,...,...,...,...,...
97,54,Нет данных,Галантерейные изделия детские из металла,Галантерейные изделия из металла,9615900000,U-образный стеклянный капилляр; Аналитические ...,,Нет данных,Нет данных
98,55,СТ РК ГОСТ Р 50962-2008,Галантерейные изделия детские из пластмасс,Галантерейные изделия детские из полимерных ма...,9615,U-образный стеклянный капилляр; Аналитические ...,,Нет данных,Нет данных
99,56,СТБ 1128-98 (ГОСТ Р 50713-94),"Постельные принадлежности (одеяла стеганые, по...",Постельные принадлежности для новорожденных из...,9404908000,маркировочное устройство; тарированный спидоме...,,Нет данных,Нет данных
100,57,СТБ 638-2001,"Одеяла, платки носовые и головные, полотенца и...",Изделия платочно-шарфовые швейные,6214300000,U-образный стеклянный капилляр; Аналитические ...,,Нет данных,Нет данных


In [87]:
m_test['Файл стандарта'] = ''

for i in range(len(m_test)):
    value = m_test['Обозначение и наименование документов в области стандартизации'].iloc[i]
    result = predict(value)
    m_test['Файл стандарта'].iloc[i] = result

C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\4071395985.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_test['Файл стандарта'].iloc[i] = result
C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\4071395985.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_test['Файл стандарта'].iloc[i] = result
C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\4071395985.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_test['Файл стандарта

In [90]:
m_test

,id,Обозначение стандарта,Группа продукции,Наименование продукции,Коды ОКПД 2 / ТН ВЭД ЕАЭС,Техническое оборудование,Укажите верно ли указано техническое оборудование?,Ваш вариант технического оборудования,Обозначение и наименование документов в области стандартизации,Файл стандарта,Должно быть по факту
0,1,"ГОСТ 10581-91, ГОСТ 32119-2013",Изделия на подкладке или без подкладки (пальто...,Одежда 3-го слоя для детей до одного года с ве...,6209909000,U-образный стеклянный капилляр; Аналитические ...,,Нет данных,Нет данных,None,
1,2,ГОСТ 19245-93,"Коляски детские, комплектующие узлы и детали к...",Коляски детские комбинированные с открытыми и ...,8715009000; 8715001000,Прибор для испытания устойчивости окраски к тр...,,Нет данных,"ГОСТ 9733.27-83 ""Материалы текстильные. Метод ...",ГОСТ 9733.27-83 Материалы текстильные. Метод и...,Прибор для испытания устойчивости окраски к тр...
2,2,ГОСТ 19245-93,"Коляски детские, комплектующие узлы и детали к...",Коляски детские комбинированные с открытыми и ...,8715009000; 8715001000,Прибор для испытания устойчивости окраски к тр...,,Нет данных,"ГОСТ 413-91 ""Ткани с резиновым или пластмассов...",ГОСТ 413-91 (ИСО 1420-87) Ткани с резиновым ил...,Стеклянный цилиндр; трубка; весы; секундомер; ...
3,3,"ГОСТ 25294-2003, ГОСТ 10581-91, ГОСТ 30386-95,...",Изделия на подкладке или без подкладки (пальто...,Изделия швейные (3-ий слой) для эпизодической ...,6204,U-образный стеклянный капилляр; Аналитические ...,,Нет данных,Нет данных,None,
4,4,ГОСТ 25295-2003,Изделия на подкладке или без подкладки (пальто...,Одежда 3-го слоя верхняя швейная пальтового ас...,6209300000,U-образный стеклянный капилляр; Аналитические ...,,Нет данных,Нет данных,None,
...,...,...,...,...,...,...,...,...,...,...,...
97,54,Нет данных,Галантерейные изделия детские из металла,Галантерейные изделия из металла,9615900000,U-образный стеклянный капилляр; Аналитические ...,,Нет данных,Нет данных,None,
98,55,СТ РК ГОСТ Р 50962-2008,Галантерейные изделия детские из пластмасс,Галантерейные изделия детские из полимерных ма...,9615,U-образный стеклянный капилляр; Аналитические ...,,Нет данных,Нет данных,None,
99,56,СТБ 1128-98 (ГОСТ Р 50713-94),"Постельные принадлежности (одеяла стеганые, по...",Постельные принадлежности для новорожденных из...,9404908000,маркировочное устройство; тарированный спидоме...,,Нет данных,Нет данных,None,
100,57,СТБ 638-2001,"Одеяла, платки носовые и головные, полотенца и...",Изделия платочно-шарфовые швейные,6214300000,U-образный стеклянный капилляр; Аналитические ...,,Нет данных,Нет данных,None,


In [ ]:
m_test['Должно быть по факту'] = ''
nlp.max_length = 1500000
for i in range(len(m_test)):
    if  m_test['Файл стандарта'].iloc[i] is not None:
        print(f"Проверяю {m_test['Группа продукции'].iloc[i]}")
        search_terms = (m_test['Техническое оборудование'].iloc[i]).split('; ')
        result = find_terms_in_text(None, search_terms, m_test['Файл стандарта'].iloc[i])
        if result != "Верно":
            m_test['Должно быть по факту'].iloc[i] = '; '.join(find_correct(m_test['Файл стандарта'].iloc[i]))
        #merged_df['Прошло проверку'].iloc[i] = result

Проверяю Коляски детские, комплектующие узлы и детали к ним


C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\2055498093.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_test['Должно быть по факту'].iloc[i] = '; '.join(find_correct(m_test['Файл стандарта'].iloc[i]))


Проверяю Коляски детские, комплектующие узлы и детали к ним


C:\Users\Lev\AppData\Local\Temp\ipykernel_10112\2055498093.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m_test['Должно быть по факту'].iloc[i] = '; '.join(find_correct(m_test['Файл стандарта'].iloc[i]))


Проверяю Детская литература, журналы и продолжающиеся издания детские


In [ ]:
m_test

In [ ]:
m_test.to_csv("m_test")

In [ ]:
g_test = m_test.groupby(['id', 'Обозначение стандарта', 'Группа продукции',
       'Наименование продукции', 'Коды ОКПД 2 / ТН ВЭД ЕАЭС',
       'Техническое оборудование'])['Должно быть по факту'].agg(list).reset_index()

In [ ]:
g_test

In [ ]:
g_test['Проверка'] = ''

# for i in range(len(merged_df)):
    
for i in range(len(g_test)):
    print(f"Проверяю {g_test['Группа продукции'].iloc[i]}")
    search_terms = (g_test['Должно быть по факту'].iloc[i])
    result = find_terms_in_text(None, search_terms, g_test['Техническое оборудование'].iloc[i])
    g_test['Проверка'].iloc[i] = result

In [ ]:
gtest2 = g_test

In [ ]:
gtest2

In [ ]:
for i in range(len(g_test)):
    g_test['Должно быть по факту'].iloc[i] = '; '.join(g_test['Должно быть по факту'].iloc[i])

In [74]:
g_test

,id,Обозначение стандарта,Группа продукции,Наименование продукции,Коды ОКПД 2 / ТН ВЭД ЕАЭС,Техническое оборудование,Должно быть по факту,Проверка
0,1,"ГОСТ 10581-91, ГОСТ 32119-2013",Изделия на подкладке или без подкладки (пальто...,Одежда 3-го слоя для детей до одного года с ве...,6209909000,U-образный стеклянный капилляр; Аналитические ...,,Верно
1,2,ГОСТ 19245-93,"Коляски детские, комплектующие узлы и детали к...",Коляски детские комбинированные с открытыми и ...,8715009000; 8715001000,Прибор для испытания устойчивости окраски к тр...,П; р; и; б; о; р; ; д; л; я; ; и; с; п; ы; т...,Неверно
2,3,"ГОСТ 25294-2003, ГОСТ 10581-91, ГОСТ 30386-95,...",Изделия на подкладке или без подкладки (пальто...,Изделия швейные (3-ий слой) для эпизодической ...,6204,U-образный стеклянный капилляр; Аналитические ...,,Верно
3,4,ГОСТ 25295-2003,Изделия на подкладке или без подкладки (пальто...,Одежда 3-го слоя верхняя швейная пальтового ас...,6209300000,U-образный стеклянный капилляр; Аналитические ...,,Верно
4,5,"ГОСТ 25295-2003, ГОСТ 25295-2003, ГОСТ 25295-2...",Изделия на подкладке или без подкладки (пальто...,Изделия 2-го слоя верхние швейные костюмного а...,6203423500; 6204623900; 6204629000,U-образный стеклянный капилляр; Аналитические ...,,Верно
5,6,"ГОСТ 29235-91 (ИСО 6742-2-85), ГОСТ 28765-90 (...",Велосипеды с высотой седла от 435 мм до 635 мм...,Велосипеды двухколесные из металла с элементам...,8712003000,маркировочное устройство; тарированный спидоме...,,Верно
6,7,ГОСТ 30407-96,"Посуда и столовые приборы (чашки, блюдца, поил...",Посуда из стеклокерамики,7013100000,pH-метр любой модели; Весы лабораторные по ГОС...,,Верно
7,8,ГОСТ 31409-2009,"Пальто, куртки и аналогичные изделия, заявленн...",Изделия верхние трикотажные третьего слоя для ...,6102209000,U-образный стеклянный капилляр; Аналитические ...,,Верно
8,9,"ГОСТ 31410-2009 , ГОСТ 31409-2009","Свитеры, джемперы, жакеты, юбки, брюки, костюм...",Изделия верхние трикотажные второго слоя для д...,6110; 6106; 6104; 6103,Хроматографическая колонка; капилляры; Весы ан...,,Верно
9,10,ГОСТ 32118-2013,Головные уборы (летние) 1-го слоя и аналогичны...,Головные уборы (летние) 1-го слоя,6505009000,U-образный стеклянный капилляр; Аналитические ...,,Верно
